In [2]:
"""
TripAdviser_Crawling version 2.1

Code to collect reviews (score rating, text review) of locations in TripAdvisor. (Apply changes according to different html, css formats for other attractions.)
"""

import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os

def parse(url):
    """Parse url."""
    #https://www.tripadvisor.co.kr/Attraction_Review-g1138011-d6847826-Reviews-Dumulmeori-Yangpyeong_gun_Gyeonggi_do.html
    url_parts = url.split('-Reviews')
    url_parts[0] += "-Reviews-or"
    url_front = url_parts[0]
    url_end = url_parts[1]
    return url_front, url_end

def openTextFile(file_loc):
    f = open(file_loc, "a", encoding="utf-8")

def crawling(url, num, 지역, loc, chrome_driver):
    """Crawl target url for target num of reviews. 지역 refers to 시도, loc refers to name of location, chrome_driver refers to location of chrome driver."""
    url_front, url_end = parse(url)
    
    data_list = []
    
    driver = webdriver.Chrome(chrome_driver) 
    """ OPTIONAL 
    # Backup storage in textfile - optional (텍스트파일에도 백업용 기록 - 필수X)
    file_loc = "file_name.txt"
    openTextFile(file_loc)
    f.write(지역+" "+loc)
    """
    
    for i in range(0,num,5): # third parameter can be set to 1
        print('Working on {}-{}'.format(i, i+5)) 
        
        url_num = str(i)
        total_url = url_front + url_num + url_end
        
        driver.get(total_url) # 위에서 url을 지정해주었다
        driver.implicitly_wait(3) # May not need wait time depending on loading speed on your computer
        time.sleep(2) # May not need wait time depending on loading speed on your computer
    
        reviews = driver.find_elements_by_css_selector(".Dq9MAugU.LnVzGwUB")
        
        page_reviews = len(reviews)

        for j in range(0,page_reviews,1): 

            #print(i)
            reviews = driver.find_elements_by_css_selector(".Dq9MAugU.LnVzGwUB") # May change for different page design
            rating_code = reviews[j].find_element_by_css_selector(".ui_bubble_rating") # May change for different page design
            cls_attr = rating_code.get_attribute("class")
            cls_attr = str(cls_attr).split("ui_bubble_rating bubble_") # May change for different page design
            score = int(cls_attr[1])

            if(j==0):
                driver.implicitly_wait(5)  # May not need wait time depending on loading speed on your computer
                driver.find_elements_by_css_selector("._3maEfNCR")[j].click() #._3maEfNCR #.ulBlueLinks

            Temp_review = reviews[j].find_element_by_css_selector(".IRsGHoPm").text#.partial_entry").text#(".partial_entry").text

            review = Temp_review.replace("\n"," ")
            """ OPTIONAL 
            # Backup storage in textfile - optional (텍스트파일에도 백업용 기록 - 필수X)
            f.write("평점:"+str(score))
            f.write("리뷰:"+review)
            f.write("\n")            
            """
            data_list.append([지역, loc, score, review])

    driver.quit()
    """ OPTIONAL 
    # Backup storage in textfile - optional (텍스트파일에도 백업용 기록 - 필수X)
    f.close()
    """
    print("DONE")
    
    return data_list
    
def dataToCSV(data_list, folder_path, file_name):
    """Save data to CSV file."""
    data = pd.DataFrame(data_list, columns=['지역', 'loc','score', 'review'])
    full_path= folder_path+file_name+'.csv'
    data.to_csv(full_path, index=False, mode='w', encoding='utf-8-sig')

def main(url, num, 지역, loc, chrome_driver, folder_path, file_name):
    data_list = crawling(url, num, 지역, loc, chrome_driver)
    dataToCSV(data_list, folder_path, file_name)

In [5]:
url = "https://www.tripadvisor.co.kr/Attraction_Review-g1138011-d6847826-Reviews-Dumulmeori-Yangpyeong_gun_Gyeonggi_do.html#REVIEWS"
num = 15
지역 = "한국"
loc = "두물머리"
chrome_driver = "C:/Users/samsung/Downloads/chromedriver_win32/chromedriver.exe"
folder_path = 'C:/Users/samsung/Desktop/'
file_name = '두물머리'
main(url, num, 지역, loc, chrome_driver, folder_path, file_name)

Working on 0-5
Working on 5-10
Working on 10-15
DONE
